In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import glob as glob
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import seaborn as sns
from pylab import rcParams

This analysis is inspired by the [ABAOJIANG's Detailed EDA](https://www.kaggle.com/code/abaojiang/jpx-detailed-eda)

### Data Specs

In [ ]:
data_specs = "/kaggle/input/jpx-tokyo-stock-exchange-prediction/data_specifications/"

In [ ]:
all_files = glob.glob(data_specs+"/*.csv")

In [ ]:
list_specs_files = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    list_specs_files.append(df)

In [ ]:
for item in all_files:
    print(item)
print(len(list_specs_files))

In [ ]:
path_main_folder = "/kaggle/input/jpx-tokyo-stock-exchange-prediction/"

# Stock List

In [ ]:
stock_list = pd.read_csv(path_main_folder+"/stock_list.csv")

In [ ]:
print(stock_list.shape)
print(f"unique stock names : {stock_list['Name'].nunique()},", f"unique security codes: {stock_list['SecuritiesCode'].nunique()}")
stock_list.head(5)

4411 < 4417. So are there any duplicates here? But security codes are unique and hence they are unique identifier of the stock list.

In [ ]:
stock_list.isnull().sum()

Let's explore other columns, meanwhile understand what do those missing values mean?

In [ ]:
for col_ in list(stock_list.columns):
    print(col_, stock_list[col_].nunique())

In [ ]:
print(stock_list['NewMarketSegment'].unique())

There are 3 foreign stocks segemnets. What does NaN meaaaaan?

In [ ]:
stock_list[stock_list['NewMarketSegment'].isna()]['Universe0'].unique()

In [ ]:
list_specs_files[4].iloc[15]['Remarks']

These stocks do not fall in the category of target universe i.e. top 2000 stocks by market capitalization. This is indicated by the Universe0 set to False for them.

In [ ]:
stock_list[stock_list['TradeDate'].isna()]['Universe0'].unique()

In [ ]:
stock_list[stock_list['Close'].isna()]['Universe0'].unique()

Same applies here. The stocks with no TradeDate or Close value are NOT in the target universe.

# Train Files - Stock Pricing Dataset

In [ ]:
train_files_path = "/kaggle/input/jpx-tokyo-stock-exchange-prediction/train_files/"

In [ ]:
stock_df = pd.read_csv(train_files_path +"/stock_prices.csv")

In [ ]:
print(stock_df.shape)
stock_df.head(3)

Notice how RowId is formed by combining Date and SecuritiesCode.

In [ ]:
stock_df['RowId'].nunique()

In [ ]:
print(f"We have the data for: {stock_df['Date'].nunique()} days and between the duration:")
print(min(stock_df['Date']) , max(stock_df['Date']))

What stocks do we have in this set and do they all have the Universe0 flag set to True in the stocks list?

In [ ]:
stocks_in_price_df = pd.Series(list(stock_df['SecuritiesCode'].unique()), name = 'Stocks_in_Price_Set')

In [ ]:
print(stocks_in_price_df.shape)
stocks_in_price_df.head(5)

In [ ]:
pd.merge(stocks_in_price_df, stock_list[['SecuritiesCode', 'Universe0']], left_on = 'Stocks_in_Price_Set', right_on = 'SecuritiesCode', how = 'left')['Universe0'].unique()

In [ ]:
round(stock_df.isnull().sum()/len(stock_df)*100, 2)

We need to look into the ExpectedDividend. Also, there are some records with missing values in 'Open', 'High', 'Close'. All these columns are pricing columns, missing values in one should mean missing values in the others.

In [ ]:
list_specs_files[2].iloc[10]['Remarks']

In [ ]:
stock_df[stock_df['ExpectedDividend'].isna()].head(10)

In [ ]:
print(stock_df[stock_df['ExpectedDividend'].isna()]['Date'].nunique())
print(min(stock_df[stock_df['ExpectedDividend'].isna()]['Date']), max(stock_df[stock_df['ExpectedDividend'].isna()]['Date']))

ExpectedDividend column is NaN for every day but not for every row (i.e. every day but not for every stock on that day).

In [ ]:
stock_df[~stock_df['ExpectedDividend'].isna()]['SecuritiesCode'].head(5)

In [ ]:
stock_day_count = stock_df.groupby('SecuritiesCode')['Date'].count().sort_values(ascending=False)
stock_day_count.head(5)

Not all stocks have records for all days in the stock price set.

In [ ]:
round(sum(stock_day_count == 1202) / len(stock_day_count), 2)*100

Stocks that have data for the entire duration comprise 93% of the stocks in the target universe list.

Now, let's look at it the other way. How many days of the total 1202 days have the data for all stocks (2000) or what percentage of those days?

In [ ]:
stock_count_on_day = stock_df.groupby('Date')['SecuritiesCode'].count().sort_values(ascending=False)

In [ ]:
print(f"Only {round(sum(stock_count_on_day == 2000)/len(stock_count_on_day)*100, 2)} percent of days have all data for all 2000 stocks.")

How many stocks have missing values of price and on which day ?

In [ ]:
stock_df[stock_df['Close'].isna()].groupby('Date')['SecuritiesCode'].count().reset_index('Date').plot(x='Date', y='SecuritiesCode',
                                                                                                     figsize=(10, 4),
                                                                                                     title = 'Count of Stocks with no price value',
                                                                                                     xlabel = 'Date',
                                                                                                     ylabel = 'Count of Stocks')
plt.grid()

In [ ]:
stock_df[stock_df['Close'].isna()].groupby('Date')['SecuritiesCode'].count().reset_index().sort_values(by='SecuritiesCode', ascending = False).iloc[0]

Number of stocks without prices has a maximum on 2020-10-01. After doing some research, I find that it's because of [The Failure of Equity Trading System on October 1, 2020.](https://www.jpx.co.jp/english/corporate/news/news-releases/0060/20201019-01.html)

Now, we explore the 'Target' and 'Volume' information in this set.

### Volume

In [ ]:
stock_df['Volume'].plot(kind='density', xlabel = 'Volume', ylabel = 'Density',
                        title='Distribution of Volume', figsize = (8, 5))
plt.grid()

Seems a right-talied distribution. But observing it altogether is not helpful. We should do it one by one for stocks in the list. But which stocks? We will pick top 5 in the frame: stock_day_count

In [ ]:
stock_day_count = stock_day_count.to_frame()
stock_day_count.reset_index(inplace=True)

In [ ]:
fig, ax = plt.subplots(nrows=5, ncols=1)

count = 0

for stock in stock_day_count[:5]['SecuritiesCode']:
    stock_df[stock_df['SecuritiesCode'] == stock]['Volume'].plot(kind = 'density', figsize = (8, 16),
                       linewidth = 1.5, ax = ax[count], label = stock)
    ax[count].legend()
    ax[count].set_xlabel("Volume")
    ax[count].set_ylabel("Density")
    ax[count].grid()
    count = count + 1

All of them have right tailed distribution.

It is good to see how the mean of the Volume varies. We will first group by the Stocks and compute mean volume for each across all their data and then plot the distribution (of mean values) in a similar manner as above.

In [ ]:
stock_mean_volume = stock_df.groupby(['SecuritiesCode'])['Volume'].mean()
stock_mean_volume = stock_mean_volume.reset_index()

ax = stock_mean_volume['Volume'].plot(kind='density', figsize=(8, 6),
                                title='Distribution of Mean Volume per stock')
ax.set_xlabel("Mean Volume")
ax.set_ylabel("Density")    
plt.grid()

In [ ]:
stock_mean_volume.mean()

Similarly, we look at the distribution of the standard deviation of the volume of stocks.

In [ ]:
stock_dev_volume = stock_df.groupby(['SecuritiesCode'])['Volume'].std()
stock_dev_volume = stock_mean_volume.reset_index()

ax = stock_dev_volume['Volume'].plot(kind='density', figsize=(8, 6),
                                title='Distribution of Standard Deviation of Volume per stock')
ax.set_xlabel("Standatd Deviation of Volume")
ax.set_ylabel("Density")    
plt.grid()

In [ ]:
stock_dev_volume.mean()

### Target

Let's observe Target column for each of of those stocks

In [ ]:
fig, ax = plt.subplots(nrows=5, ncols=1)

count = 0

for stock in stock_day_count[:5]['SecuritiesCode']:
    stock_df[stock_df['SecuritiesCode'] == stock]['Target'].plot.density(figsize = (8, 16),
                       linewidth = 1.5, ax = ax[count], label = stock)
    ax[count].legend()
    ax[count].grid()
    count = count + 1

For the same stocks in the list, the Target column closely resembles normal distribution with mean equal to 0.

Let us look at the distribution of the mean of Target value

In [ ]:
stock_mean_target = stock_df.groupby(['SecuritiesCode'])['Target'].mean()
stock_mean_target = stock_mean_target.reset_index()

ax = stock_mean_target['Target'].plot(kind='density', figsize=(8, 6),
                                title='Distribution of Mean Target per stock')
ax.set_xlabel("Mean Target")
ax.set_ylabel("Density")    
plt.grid()

In [ ]:
stock_mean_target.mean()

This is right-tailed distribution as well.

In [ ]:
stock_dev_target = stock_df.groupby(['SecuritiesCode'])['Target'].std()
stock_dev_target = stock_mean_target.reset_index()

ax = stock_dev_target['Target'].plot(kind='density', figsize=(8, 6),
                                title='Distribution of Standard Deviation of Target per stock')
ax.set_xlabel("Standard Deviation of Target")
ax.set_ylabel("Density")    
plt.grid()

In [ ]:
stock_dev_target.mean()

In [ ]:
def plot_candle_with_target(stock_code):
    
    """Plot OHLCV plot with target series.
    
    Parameters:
        stock_code: int, code of the stock
    """
    
    df_ = stock_df[stock_df['SecuritiesCode'] == stock_code]
    dates = df_['Date'].values
    ohlc = {
        'open': df_['Open'].values, 
        'high': df_['High'].values, 
        'low': df_['Low'].values, 
        'close': df_['Close'].values
    }
    vol = df_['Volume'].values
    target = df_['Target'].values
    
    fig = make_subplots(rows=3, cols=1, shared_xaxes=True, x_title='Date')
    fig.add_trace(go.Candlestick(x=dates, name='OHLC', **ohlc),
                  row=1, col=1)
    fig.add_trace(go.Bar(x=dates, y=vol, name='Volume'),
                  row=2, col=1)
    fig.add_trace(go.Scatter(x=dates, y=target, name='Target'),
                  row=3, col=1)
    fig.update_layout(
        title=f"OHLCV Chart with Target Series (Stock {stock_code})",
    )
    fig.update(layout_xaxis_rangeslider_visible=False)
    fig.show()

In [ ]:
for stock_code in stock_day_count['SecuritiesCode'][:5]:
    plot_candle_with_target(stock_code)

#### Let us conduct analysis on the Stock list before we merge with Stock prices for enahnced analysis

In [ ]:
stock_list.head(5)

In [ ]:
stock_list = stock_list[stock_list['Universe0'] == True]

In [ ]:
marketsegments = stock_list['NewMarketSegment'].value_counts(sort = True)

colors = ["blue", "green", "red"]
explode = (0, 0.1, 0.1)  

labels = marketsegments.index.to_list()
 
rcParams['figure.figsize'] = 6, 6
plt.rcParams['text.color'] = 'white'

# Plot
plt.pie(marketsegments, explode=explode, labels=labels, colors=colors,
        autopct='%1.1f%%', shadow=True, startangle=270,)

plt.rcParams['font.size'] = 12

plt.title('Pie Chart on Distribution of NewMarketSegment in stock list',size = 14)
plt.show()

In [ ]:
ax = stock_list['17SectorName'].value_counts().plot(kind='bar', figsize=(10, 5),
                                    title="Bar chart for 17SectorName of Stocks")
ax.set_xlabel("17SectorName", fontsize=14)
ax.set_ylabel("Count of Stocks", fontsize=14)
plt.tick_params(labelsize=12)
#plt.xticks(rotation = 45)
plt.grid()
plt.show()

In [ ]:
ax = stock_list['33SectorName'].value_counts().plot(kind='bar', figsize=(14, 5))
ax.set_xlabel("33SectorName", fontsize=14)
ax.set_ylabel("Count of Stocks", fontsize=14)
ax.set_title("Bar chart for 33SectorName of Stocks")
plt.tick_params(labelsize=12)
plt.grid()
plt.show()

In [ ]:
sections = stock_list['Section/Products'].value_counts(sort = True)

colors = ["blue", "green", "red", "orange", "purple"]
explode = [0.1, 0.1, 0.1, 0.1, 0.2]

labels = sections.index.to_list()
 
rcParams['figure.figsize'] = 6, 6
plt.rcParams['text.color'] = 'white'

# Plot
plt.pie(sections, explode=explode, labels=labels, colors=colors,
        autopct='%1.1f%%', shadow=True, startangle=270,)

plt.rcParams['font.size'] = 12

plt.title('Pie Chart on Distribution of NewMarketSegment in stock list',size = 14)
plt.show()

In [ ]:
rich_stock_df = pd.merge(stock_df[['SecuritiesCode', 'Close', 'Target', 'Volume']], stock_list[['SecuritiesCode', 'Section/Products', 'NewMarketSegment', '33SectorName', '17SectorName']], on = 'SecuritiesCode')

In [ ]:
print(rich_stock_df.shape)
rich_stock_df.head(3)

Let us check mean Volume and Target for each of 33SectorName

In [ ]:
plt.rcParams['text.color'] = 'black'

In [ ]:
fig, ax = plt.subplots(nrows=5, ncols=1)

count = 0

for stock in stock_day_count[:5]['SecuritiesCode']:
    stock_df[stock_df['SecuritiesCode'] == stock]['Target'].plot.density(figsize = (8, 16),
                       linewidth = 1.5, ax = ax[count],label = stock)
    ax[count].legend()
    ax[count].grid()
    count = count + 1

We want to analyze the mean value of Target, Volume and Closing Price of each 33SectorName in the Stock list.

In [ ]:
fig, ax = plt.subplots(3, 1, sharex=True)

count = 0

for var in ['Target', 'Volume', 'Close']:
    rich_stock_df.groupby('33SectorName')[var].mean().plot(kind='bar', label = var,
                                                           xlabel='33SectorName',
                                                           ylabel=f"Mean {var} Value",
                                                           figsize = (14, 7), ax = ax[count])
    ax[count].legend()
    ax[count].grid()
    count = count + 1

* Banks have negative mean Target value
* Banks also have the highest mean of Volume
* Marine Transportation, Information & Communication, and Services have very high mean Target value compared to others

We can repeat the same analysis on Sections/Products

In [ ]:
fig, ax = plt.subplots(3, 1, sharex=True)

count = 0

for var in ['Target', 'Volume', 'Close']:
    rich_stock_df.groupby('Section/Products')[var].mean().plot(kind='bar', label = var,
                                                           xlabel='Section/Products',
                                                           ylabel=f"Mean {var} Value",
                                                           figsize = (12, 7), ax = ax[count])
    ax[count].legend()
    ax[count].grid()
    ax[count].tick_params(axis='x', labelrotation = 45)

    count = count + 1

Let us check mean Target value for each of NewMarketSegment

In [ ]:
rich_stock_df.groupby('NewMarketSegment')['Target'].mean().sort_values(ascending=False).plot(kind='bar',
                                                                                        title='Bar plot of Mean Target Value of NewMarketSegment',
                                                                                        xlabel='NewMarketSegment',
                                                                                        ylabel='Mean Target Value',
                                                                                        figsize = (6, 4))
plt.grid()

Growth Market has the highest Target mean in the stock pricing dataset

In [ ]:
There are more data files to analyze in the directory. However, we limit the scope of this notebook to Stock Pricing only.

any